In [1]:
from azureml.core import Workspace, Datastore, Experiment, Environment
from azureml.core.runconfig import RunConfiguration
from azureml.pipeline.steps import PythonScriptStep
from azureml.pipeline.core import Pipeline

In [2]:
pip install azureml-pipeline

Note: you may need to restart the kernel to use updated packages.


In [3]:
# Initialize workspace
ws = Workspace.from_config("Users/mypersonall3099/Online_retail/config.json")

# Define compute target
compute_name = "OnlineRetail"
compute_target = ws.compute_targets[compute_name]

In [4]:
#vm_size = "Standard_E4ds_v4"

# Define environment
env = Environment.from_conda_specification(name="myenv", file_path="environment.yaml")
env.register(workspace=ws)


{
    "assetId": "azureml://locations/eastus2/workspaces/7ce76f32-f5fe-4dd3-b900-96fb92b58ebb/environments/myenv/versions/3",
    "databricks": {
        "eggLibraries": [],
        "jarLibraries": [],
        "mavenLibraries": [],
        "pypiLibraries": [],
        "rcranLibraries": []
    },
    "docker": {
        "arguments": [],
        "baseDockerfile": null,
        "baseImage": "mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04:20240709.v1",
        "baseImageRegistry": {
            "address": null,
            "password": null,
            "registryIdentity": null,
            "username": null
        },
        "buildContext": null,
        "enabled": false,
        "platform": {
            "architecture": "amd64",
            "os": "Linux"
        },
        "sharedVolumes": true,
        "shmSize": null
    },
    "environmentVariables": {
        "EXAMPLE_ENV_VAR": "EXAMPLE_VALUE"
    },
    "inferencingStackVersion": null,
    "name": "myenv",
    "python": {
       

In [27]:
from azureml.core import Environment

env = Environment.get(workspace=ws, name="myenv")
print(env.python.conda_dependencies.serialize_to_string())


name: myenv
channels:
- defaults
dependencies:
- python=3.7
- scikit-learn
- statsmodels
- pip
- pip:
  - pmdarima
  - azureml-sdk
  - azureml-dataset-runtime[fuse, pandas]
  - joblib
  - scikit-learn



In [28]:
#Declaring environment
# aml_config = RunConfiguration()
# aml_config.target = compute_target 

# #env = Environment.from_conda_specification(name="myenv", file_path="environment.yaml")


# Define run configuration
aml_config = RunConfiguration()
aml_config.target = compute_target
aml_config.environment = env

In [29]:
#Pipeline
read_data = 'data_wrangling.py'
prep = 'preprocessing.py'
model = 'modelling.py'

source_directory = ""

#Script initialization
py_script_run_read = PythonScriptStep(
                name='Data Wrangling Step',
                script_name = read_data,
                compute_target = compute_target,
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_prep = PythonScriptStep(
                script_name = prep,
                compute_target=compute_target,
                source_directory=source_directory,
                runconfig = aml_config,
                allow_reuse=False)

py_script_run_model = PythonScriptStep(
                script_name = model,
                compute_target=compute_target,
                source_directory=source_directory,
                runconfig = aml_config,
                allow_reuse=False)

pipeline_steps = [py_script_run_read, py_script_run_prep, py_script_run_model]
pipeline_1 = Pipeline(workspace=ws, steps = pipeline_steps)

In [30]:
# # Define pipeline steps
# read_data = 'data_wrangling.py'
# prep = 'preprocessing.py'
# model = 'modelling.py'

# py_script_run_read = PythonScriptStep(
#     script_name=read_data,
#     compute_target=compute_target,
#     arguments=['--input-data', 'online_retail_utf8.csv'],
#     runconfig=aml_config,
#     allow_reuse=False
# )

# py_script_run_prep = PythonScriptStep(
#     script_name=prep,
#     compute_target=compute_target,
#     arguments=['--prep', 'wrangled.csv'],
#     runconfig=aml_config,
#     allow_reuse=False
# )

# py_script_run_model = PythonScriptStep(
#     script_name=model,
#     compute_target=compute_target,
#     arguments=['--train', 'preprocessed.csv'],
#     runconfig=aml_config,
#     allow_reuse=False
# )

# # Create and submit pipeline
# pipeline_steps = [py_script_run_read, py_script_run_prep, py_script_run_model]
# pipeline = Pipeline(workspace=ws, steps=pipeline_steps)


In [31]:
# Experiment

# pipeline_run = Experiment(ws, "First_run_7").submit(pipeline_1)
# pipeline_run.wait_for_completion(show_output=True)

experiment_name = "Run3"
experiment = Experiment(ws, experiment_name)
pipeline_run = experiment.submit(pipeline_1)
pipeline_run.wait_for_completion(show_output=True)

Created step Data Wrangling Step [ca11af94][2e267005-6925-43e9-b67a-6d35914725ed], (This step will run and generate new outputs)Created step preprocessing.py [2667c172][3d67fa4c-eaee-4776-8210-40a9334c6def], (This step will run and generate new outputs)

Created step modelling.py [13784c78][a8199df6-9538-495f-8dc7-ac8f7c3bc60b], (This step will run and generate new outputs)
Submitted PipelineRun fb28e581-f117-40d0-afa0-4dba5682226a
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/fb28e581-f117-40d0-afa0-4dba5682226a?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRunId: fb28e581-f117-40d0-afa0-4dba5682226a
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/fb28e581-f117-40d0-afa0-4dba5682226a?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
Pipeline

'Finished'

In [32]:
# Experiment 2

pipeline_run = Experiment(ws, "Second_run").submit(pipeline_1)
pipeline_run.wait_for_completion(show_output=True)

Submitted PipelineRun 42601c31-040e-4aac-91b3-8ed00e0e80fa
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/42601c31-040e-4aac-91b3-8ed00e0e80fa?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRunId: 42601c31-040e-4aac-91b3-8ed00e0e80fa
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/42601c31-040e-4aac-91b3-8ed00e0e80fa?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
PipelineRun Status: NotStarted
PipelineRun Status: Running


StepRunId: 0592e3a7-ddda-4f2d-8279-8df845a155d9
Link to Azure Machine Learning Portal: https://ml.azure.com/runs/0592e3a7-ddda-4f2d-8279-8df845a155d9?wsid=/subscriptions/d8c4fe8f-f720-48ac-86e8-880576d9cd7a/resourcegroups/OnlineRetail/workspaces/OnlineRetail&tid=f56f1f69-458e-427b-bada-4cba658f7917
StepRun( Data Wrangling 

'Finished'

In [33]:
pipeline_run

Experiment,Id,Type,Status,Details Page,Docs Page
Second_run,42601c31-040e-4aac-91b3-8ed00e0e80fa,azureml.PipelineRun,Completed,Link to Azure Machine Learning studio,Link to Documentation


In [35]:
datastore = Datastore.get(ws, 'workspaceblobstore')
datastore.download(target_path="azureml", prefix="", overwrite=False, show_progress=True)

Downloaded 52ab5d18-b7c7-5c08-a45d-e05c04af9ad4/evaluate_model.yaml, 1 files out of an estimated total of 2202
Downloaded 52ab5d18-b7c7-5c08-a45d-e05c04af9ad4/invoker.py, 2 files out of an estimated total of 2202
Downloaded 52ab5d18-b7c7-5c08-a45d-e05c04af9ad4/urldecode_invoker.py, 3 files out of an estimated total of 2202
Downloaded 56d1c1b4-67cd-50ab-8702-374b3afaa321/invoker.py, 4 files out of an estimated total of 2202
Downloaded 56d1c1b4-67cd-50ab-8702-374b3afaa321/select_columns_in_dataset.yaml, 5 files out of an estimated total of 2202
Downloaded 56d1c1b4-67cd-50ab-8702-374b3afaa321/urldecode_invoker.py, 6 files out of an estimated total of 2202
Downloaded 6cdc770f-40e8-5210-926c-a855e7839970/execute_python_script.yaml, 7 files out of an estimated total of 2202
Downloaded 6cdc770f-40e8-5210-926c-a855e7839970/invoker.py, 8 files out of an estimated total of 2202
Downloaded 6cdc770f-40e8-5210-926c-a855e7839970/urldecode_invoker.py, 9 files out of an estimated total of 2202
Downloa

2202